In [0]:
# pyspark functions
from pyspark.sql.functions import *

In [0]:
# file location and type
#aAsterisk(*) indicates reading all the content of the specified file that have .json extension
pin_location = "/mnt/data/topics/0affea73130b.pin/partition=0/*.json" 
file_type = "json"
# ask Spark to infer the schema
infer_schema = "true"
# read in JSONs from mounted S3 bucket
pin_df = spark.read.format(file_type).option("inferSchema", infer_schema).load(pin_location)

In [0]:
cleaned_df = pin_df.cache()

In [0]:
# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# replace empty values with none
cleaned_df = cleaned_df.replace({'No description available Story format': None}, subset=['description'])
cleaned_df = cleaned_df.replace({'User Info Error': None}, subset=['follower_count'])
cleaned_df = cleaned_df.replace({'Image src error.': None}, subset=['image_src'])
cleaned_df = cleaned_df.replace({'User Info Error': None}, subset=['poster_name'])
cleaned_df = cleaned_df.replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None}, subset=['tag_list'])
cleaned_df = cleaned_df.replace({'No Title Data Available': None}, subset=['title'])

# cast int data type to follower_count column
cleaned_df = cleaned_df.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
cleaned_df = cleaned_df.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
cleaned_df = cleaned_df.withColumn("follower_count", cleaned_df["follower_count"].cast("int"))

# cast int data type to numeric data
cleaned_df = cleaned_df.withColumn("downloaded", cleaned_df["downloaded"].cast("int"))
cleaned_df = cleaned_df.withColumn("index", cleaned_df["index"].cast("int"))

# clean save_location column to only include directory path
cleaned_df = cleaned_df.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# rename index to ind
cleaned_df = cleaned_df.withColumnRenamed("index", "ind")

# drop duplicates
cleaned_df = cleaned_df.dropDuplicates()

# reorder columns
cleaned_df = cleaned_df.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")

In [0]:
pin_df = cleaned_df.cache()